In [1]:
import torch
from torch import nn
from src.data.make_housing_dataset import CaliforniaHousing
from torchvision.utils import make_grid
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import seaborn as sns
from src.models.regFNN import RegFNN
from sklearn import metrics
import numpy as np
import torch.nn.functional as nnf
from skimage import io
from src.models.CQR import CQR
from src.utils.cp_reg_utils import *
from src.models.conformal_model import ConformalModel
# from pytorch_forecasting import QuantileLoss



In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
test_set = CaliforniaHousing(
    split="test", in_folder='../data/raw', out_folder='../data/processed')
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)
calib_set = CaliforniaHousing(
    split="calib", in_folder='../data/raw', out_folder='../data/processed')
calib_loader = torch.utils.data.DataLoader(calib_set, batch_size=32, shuffle=True)


Loaded from pre-processed files
Loaded from pre-processed files


In [4]:
model = RegFNN()
model.load_state_dict(torch.load("../models/trained_reg_fnn.pt"))

<All keys matched successfully>

In [5]:
cmodel = CQR(model, calib_loader, alpha=0.1)

Computing logits for model (only happens once).


100%|██████████| 33/33 [00:00<00:00, 564.46it/s]


In [9]:
for input, target in test_loader:
    out = cmodel(input)
    print(out[:,0])
    break

TypeError: list indices must be integers or slices, not tuple

In [6]:
validate(test_loader, cmodel, print_bool=True)

tensor([[1.6263, 4.4881],
        [1.0364, 3.2022],
        [1.0424, 3.3751],
        [0.5602, 1.9532],
        [0.9046, 2.9899],
        [1.1723, 3.5321],
        [1.3530, 3.8695],
        [0.8280, 2.8652],
        [0.8230, 2.5792],
        [0.6000, 2.0931],
        [0.9986, 3.2393],
        [1.5359, 4.3589],
        [1.0876, 3.2851],
        [0.7004, 2.5781],
        [0.7637, 2.4939],
        [1.3366, 3.9273],
        [0.6121, 2.0644],
        [0.5705, 2.0079],
        [1.4566, 4.2271],
        [0.7703, 2.5424],
        [0.9450, 2.8548],
        [1.2984, 3.8893],
        [0.5496, 2.3964],
        [0.9761, 3.2398],
        [1.1141, 3.5934],
        [1.3397, 4.4321],
        [1.0097, 3.1366],
        [1.3957, 4.0825],
        [0.7983, 2.9338],
        [0.7066, 2.3802],
        [0.8592, 2.8146],
        [0.8652, 2.6530]])
N: 32 | Time: 0.006 (0.006) | Length: 3.688 (3.688) | Cov: 1.000 (1.000)tensor([[0.9073, 2.7705],
        [1.2054, 3.4863],
        [1.2091, 3.6006],
        [0.8439, 

(3.7781967673190806, 0.9772286821705426)

In [7]:
cmodel.Qhat

0.7745585441589355

In [11]:
with torch.no_grad():
    model.eval()
    E = np.array([])
    # accuracies = []
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        output = cmodel(inputs)
        print(output)
        # accuracies.append(metrics.r2_score(targets.detach().cpu(), output.detach().cpu()[:,1]))
        break
    model.train()
# print(np.mean(accuracies))

[array([ 0.12084001, -0.21709043, -0.23587763,  0.3195827 ,  0.4605925 ,
        0.2433697 ,  0.9891541 ,  0.41318107, -0.03734404, -0.12084353,
       -0.07658076,  0.35477436, -0.06228042,  1.2413249 ,  0.1253165 ,
        0.01011562,  0.29953527, -0.09795558, -0.04394138,  0.42062497,
        0.2656741 ,  0.48682475,  1.2335465 ,  0.38536263, -0.03006744,
        0.684919  ,  0.01548165,  0.43516505, -0.07386881, -0.12464011,
       -0.10574555,  0.26648128], dtype=float32), array([3.9736466, 2.691869 , 2.64404  , 4.1674747, 4.510119 , 3.8817863,
       5.7549086, 4.3298254, 3.3471515, 3.0846462, 3.0886045, 4.0075808,
       3.4807498, 6.060411 , 3.9157548, 3.9958339, 4.1753407, 3.2810783,
       3.1382551, 4.4861193, 4.506222 , 4.6065087, 6.295882 , 4.1390004,
       3.1858253, 4.907175 , 3.3106482, 4.3358564, 3.5478868, 3.1694036,
       3.0319748, 3.8943238], dtype=float32)]


In [13]:
test_set.targets

tensor([1.6560, 1.2610, 0.9440,  ..., 0.9230, 0.8470, 0.8940],
       dtype=torch.float64)

In [16]:
pred = np.array([-1,2,4,6,-3])
tar = np.array([1,9,2,4,-3])

In [20]:
np.abs(pred - tar)

array([2, 7, 2, 2, 0])

In [10]:
alpha = 0.1
1-(alpha/2)

0.95

In [12]:
n=1032
np.ceil((n+1)*(1-alpha))/n

0.9011627906976745

In [9]:
validate_CQ(test_loader, model, True)

N: 4128 | Time: 0.101 (0.056) | Length: 2.317 (2.229) | Cov: 0.875 (0.839)


In [11]:
len(test_set)

4128

In [12]:
len(calib_loader.dataset)

1032